In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
sys.path.append('/Your/Location/To/CEECNetRepo/')

In [3]:
from mxnet import nd 
from ceecnet.models.changedetection.mantis.mantis_dn import *

/usr/local/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 28] No space left on device.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [4]:
# D6nf32 example 
depth=6
norm_type='GroupNorm'
norm_groups=4
ftdepth=5
NClasses=2
nfilters_init=32
psp_depth=4
nheads_start=4


net = mantis_dn_cmtsk(nfilters_init=nfilters_init, NClasses=NClasses,depth=depth, ftdepth=ftdepth, model='CEECNetV1',psp_depth=psp_depth,norm_type=norm_type,norm_groups=norm_groups,nheads_start=nheads_start)
net.initialize()

depth:= 0, nfilters: 32, nheads::4, widths::1
depth:= 1, nfilters: 64, nheads::8, widths::1
depth:= 2, nfilters: 128, nheads::16, widths::1
depth:= 3, nfilters: 256, nheads::32, widths::1
depth:= 4, nfilters: 512, nheads::64, widths::1
depth:= 5, nfilters: 1024, nheads::128, widths::1
depth:= 6, nfilters: 512, nheads::128, widths::1
depth:= 7, nfilters: 256, nheads::64, widths::1
depth:= 8, nfilters: 128, nheads::32, widths::1
depth:= 9, nfilters: 64, nheads::16, widths::1
depth:= 10, nfilters: 32, nheads::8, widths::1


In [5]:
BatchSize = 4
img_size=256
NChannels = 3

input_img_1 = nd.random.uniform(shape=[BatchSize, NChannels, img_size, img_size])
input_img_2 = nd.random.uniform(shape=[BatchSize, NChannels, img_size, img_size])

## Example of forward operation:


In [6]:
outs = net(input_img_1, input_img_2)

In [7]:
# outs is a list of outputs, segmentation, boundary, distance. 
# Each has shape BatchSize, NClasses, img_size, img_size
for out in outs:
    print (out.shape)

(4, 2, 256, 256)
(4, 2, 256, 256)
(4, 2, 256, 256)


### Example of performing backward with multitasking operation

In [8]:
labels_segm = nd.random.uniform(shape=[BatchSize, NClasses, img_size, img_size])
labels_segm = labels_segm > 0.5

labels_bound = nd.random.uniform(shape=[BatchSize, NClasses, img_size, img_size])
labels_bound = labels_bound > 0.5

labels_dist = nd.random.uniform(shape=[BatchSize, NClasses, img_size, img_size])


labels = [labels_segm,labels_bound,labels_dist]
labels = nd.concat(*labels,dim=1)

In [9]:
from mxnet import autograd
from ceecnet.nn.loss.mtsk_loss import *

In [10]:
myMTSKL = mtsk_loss()

In [12]:
with autograd.record():
    listOfPreds = net(input_img_1, input_img_2)
    loss = myMTSKL.loss(listOfPreds,labels)
    loss.backward()

In [13]:
loss.shape

(4,)

In [28]:
loss


[0.50219935 0.5020496  0.5023406  0.5021815 ]
<NDArray 4 @cpu(0)>